# 环境介绍
本示例运行在windows，使用cpu

# 本示例概要
本示例主要是带领大家如何讓本地端建置的LLM可以跟PDF或是docx等等文件格式進行互動，這範例的應用可以是在企業裡可以提供客戶支持或教育應用等等，那我们接着来实作吧。一下範例在之前的範例有安裝過，在這邊就不額外說明，請參考之前的範例安裝說明。


# 范例说明

1️⃣ 在Anaconda上安装相关套件


> pip install unstructured

2️⃣ 范例代码


> 首先我们先把页面的代码进行撰写，并透过streamlit来查看页面（在脚本中不要使用"""说明"""这种备注，因为在页面中会出现你备注的文字，所以我这里备注都采取#）



In [ ]:
#強制你的腳本接著啟動 Ollama 伺服端（或該進程中載入的其他深度學習庫）時，就會以 CPU 模式運作。如果你是GPU可以拿掉此代碼
# import os
# import subprocess

# # 設定環境變數，確保 Ollama 伺服端以 CPU 模式啟動
# env = os.environ.copy()
# env["CUDA_VISIBLE_DEVICES"] = "-1"

# # 啟動 Ollama 伺服端
# subprocess.Popen(["ollama", "serve"], env=env)

import streamlit as st
import os
import time
#导入相关语言链接库模型
from langchain.prompts import PromptTemplate
#PromptTemplate 則是幫你把「提示模板 (prompt template)」和「動態參數」結合起來，做到可重複、可維護、可彈性插入上下文的功能。
from langchain.memory import ConversationBufferMemory

# 記憶體 (Memory) 模組主要功能是在對話過程中，能夠紀錄並回傳歷史對話內容或特定資訊，讓模型在後續回合時可以參考先前上下文。
# 常見的幾種記憶體類型：
# ConversationBufferMemory

# 將對話過程「完整」地存起來（類似累積的聊天紀錄）。
# 在下一次呼叫 Chain（或模型）時，會把所有對話紀錄（或你指定要包含的內容）注入 Prompt 中。
# 特性：實作簡單，但隨著對話輪數增加，Prompt 也會變得非常長。可能會有 Token 用量限制的問題。
# ConversationBufferWindowMemory

# 跟 ConversationBufferMemory 類似，但只保留「最近 N 輪」的對話內容，避免 Prompt 過度冗長。
# 適用於對話過程較長時，可以設定一個「視窗大小 (window size)」，只把最近幾輪對話放進 Prompt。
# ConversationSummaryMemory

# 隨著對話進行，會把先前對話歸納成更精簡的摘要。
# 下次呼叫模型時，給模型的提示就不會是全部對話文字，而是已摘要過的內容。
# 適合長對話的場景，既能保留上下文意義，又不會導致 Token 過量。
# ConversationKGMemory

# 透過知識圖譜 (Knowledge Graph) 的形式，動態記錄對話中提到的人、事、物，以及它們之間的關係。
# 更適合需要複雜推理、人物/事件追蹤的應用。

#官方
#from langchain.vectorstores import Chroma
#社群版
from langchain_community.vectorstores.chroma import Chroma


# Chroma 是一款開源的向量資料庫（或嵌入式向量存儲），能夠將你的文本資料轉為向量後儲存，再使用相似度檢索的方式找到最相關的片段。
# chroma 這個類別或函式，就代表你要使用 Chroma 來保存與檢索文本向量，用於 RAG（檢索增強生成）流程中。

from langchain_community.embeddings.ollama import OllamaEmbeddings

# 表示你正在使用社群版的 LangChain 擴充包，來支援 Ollama 這類模型做 Embeddings（向量化）。

# Ollama 本身是一種可以在本地端推理的模型或工具，具備產生向量嵌入的能力。
# 這種擴充包通常是社群貢獻，讓 LangChain 更加多元，無論是開源大模型還是商業 API，都能整合。

from langchain_community.llms import Ollama
#使用 LangChain 社群實作的某個 LLM 接口，與 Ollama 搭配。
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# LangChain 也提供「Callback」功能，可以在執行 Chain 或呼叫模型的每個步驟時，觸發特定事件。

# 例如 StreamingStdOutCallbackHandler 可以讓 LLM 產生內容時，實時地在終端機（console）中呈現文字，方便即時觀察模型的回應過程。
# 這對於除錯、監控或做互動式應用（例如 Chatbot）特別有用。

from langchain.callbacks.manager import CallbackManager


# 在 LangChain 的生態系中，CallbackManager 負責「管理整個執行流程中的回呼 (callback)」，也就是在某些關鍵事件（例如模型產生 Token、Chain 開始或結束、Agent 做出決策等）時，執行特定的回呼函式（callback function）。這些回呼可以用來做各式各樣的事情，例如：

# 即時監控：在產生 Token 或 Chain 執行時，馬上將中間結果輸出到前端介面，實現串流式回應 (streaming)。
# 記錄 (logging) 或除錯 (debug)：將執行過程中發生的事件紀錄下來，方便後續檢查與除錯。
# 自訂行為：在每個事件點注入額外的動作，例如寫入資料庫、更新進度條、觸發通知等等。
from langchain_community.document_loaders import (PyPDFLoader,UnstructuredWordDocumentLoader,UnstructuredPowerPointLoader,UnstructuredFileLoader)

#主要用於將 文件轉換成可被大模型或語言處理管線所用的結構化文本
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 在 LangChain 中扮演文件前處理的重要角色，能智能地切分文本，讓語言模型在處理或檢索時更有效率且具可讀性。
# 它採用「從大到小、遞迴式」的方式逐層切分，盡量保留原文的語義結構，同時尊重 chunk_size 的限制。
# 在實際應用中，你可以根據不同文件類型（如技術文檔、聊天記錄、新聞稿、小說等）的特性，自訂分隔符與切分參數，並持續迭代測試，以找到最適合的切分策略。

from langchain.chains import RetrievalQA

# RetrievalQA 是 LangChain 提供的一個高階 Chain，旨在把「檢索外部資料」與「透過 LLM 生成答案」這兩個核心過程一站式整合，非常適合用於需要「文件問答」或「根據外部資料產生回應」的應用場景。
# 它完美體現了 RAG (Retrieval-Augmented Generation) 的思路：先檢索，後生成，確保回答更加準確且有根據，而不只是依賴模型的「內建知識」。
# 只要你準備好合適的向量資料庫（或其他檢索機制）及LLM，再使用幾行程式碼就能迅速搭建一個檢索式問答服務。


#確保文件夾存在，若不存在則創建
pdf_directory ='pdfFiles'
vector_directory = 'vectorDB'
os.makedirs(pdf_directory,exist_ok=True)
os.makedirs(vector_directory,exist_ok=True)

#初始化 Streamlit 會話狀態，定義聊天機器人的提示模板,定義了機器人如何回答使用者的模板。
if 'chat_template' not in st.session_state:
    st.session_state.chat_template="""您是一個協助解答文檔中問題的聊天機器人，根據context回答問題。若無法回答則表示'根據資料無法回答此問題'。

    Context: {context}
    History: {history}
    User: {question}
    Chatbot:"""

#建立 PromptTemplate，指定prompt接受的格式,將上面的提示模板與歷史對話紀錄、內容及使用者問題進行結合。
if 'prompt_template' not in st.session_state:
    st.session_state.prompt_template = PromptTemplate(
        input_variables=["history","context","question"],
        template = st.session_state.chat_template,
        )

#指定上下文存儲的範,用於儲存歷史聊天記錄。ConversationBufferMemory可以參考上面備註
if 'conversation_memory' not in st.session_state:
    st.session_state.conversation_memory=ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question",
        )

#建立向量資料庫 Chroma，指定專程向量的編碼。建立 Chroma 資料庫並設定 embedding 模型為 Ollama。
if 'embeddings_store' not in st.session_state:
    st.session_state.embeddings_store=Chroma(
        persist_directory=vector_directory,
        embedding_function=OllamaEmbeddings(
            base_url='http://localhost:11434',
            model="mxbai-embed-large"
            )
        )


#指定語言模型

# 這個範例使用的模型可以考慮切換對中文適洽較高的模型

# 「回呼（callback）」可以想像成事件觸發後要去執行的動作。例如模型在一步一步生成文字的過程中，程式可以在每產生一段字串時「呼叫」一些自訂的函式來處理或顯示這些資訊。

# CallbackManager：一個可以放多個「回呼處理器（callback handlers）」的管理器。當模型產生新的文字或完成某些階段時，這個管理器就會呼叫其中所有的回呼處理器。
# StreamingStdOutCallbackHandler()：一個專門用來在推論過程中「即時」把文字輸出到畫面（或終端機）的回呼處理器。它收到新產生的文字時，就會立刻把這些文字印到標準輸出（終端機）。
#建立 Ollama 語言模型,使用 Ollama 的 llama3 模型作為主要大模型，提供流式回覆功能。
if 'language_model' not in st.session_state:
    st.session_state.language_model = Ollama(base_url="http://localhost:11434",
                                             model="llama3",
                                             verbose=True,
                                             callback_manager=CallbackManager(
                                                 [StreamingStdOutCallbackHandler()]),
                                             )
#用來「記住」某些資料或狀態，避免每次重新執行都把之前的內容遺失掉。
if 'chat_history' not in st.session_state:
    st.session_state.chat_history=[]


#主標題
st.title("Chatbot With More Files")

#PDF 文件上傳器
uploaded_file = st.file_uploader("請選擇檔案", type=["pdf", "docx", "pptx", "txt"])

#顯示聊天歷史
for message in st.session_state.chat_history:
    with st.chat_message(message["role"]):
        st.markdown(message["message"])


#處理上傳的PDF文件，對於chunk_size和chunk_overlap可以根據你的硬體需求以及你對解析的精準度作為調整
# 處理上傳的 PDF 文件
if uploaded_file is not None:
    file_path = os.path.join(pdf_directory, uploaded_file.name)
    if not os.path.exists(file_path):
        with st.status("Saving file..."):
            with open(file_path, 'wb') as f:
                f.write(uploaded_file.getvalue())

            file_extension = uploaded_file.name.split(".")[-1]
            if file_extension.lower() == "pdf":
                # 處理PDF檔案的程式碼
                loader = PyPDFLoader(file_path)
            elif file_extension.lower() == "docx":
                # 處理DOCX檔案的程式碼
                loader = UnstructuredWordDocumentLoader(file_path)
            elif file_extension.lower() == "pptx":
                # 處理PPTX檔案的程式碼
                loader = UnstructuredPowerPointLoader(file_path)
            elif file_extension.lower() == "txt":
                # 處理TXT檔案的程式碼
                loader = UnstructuredFileLoader(file_path)


            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1500,
                chunk_overlap=200,
                length_function=len
            )

            doc = loader.load_and_split(text_splitter)

            st.session_state.embeddings_store = Chroma.from_documents(
                documents=doc,
                embedding=OllamaEmbeddings(model="mxbai-embed-large")
            )
            st.session_state.embeddings_store.persist()


    #啟動PDF文檔檢索
    st.session_state.retriever = st.session_state.embeddings_store.as_retriever()

    #啟動 PDF 檢索器與問答鏈 (QA Chain),建立能根據使用者提問從 PDF 中找到答案的問答鏈。
    if 'qa_chain' not in st.session_state:
        st.session_state.qa_chain = RetrievalQA.from_chain_type(
            llm=st.session_state.language_model,
            chain_type='stuff',
            retriever=st.session_state.retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt":st.session_state.prompt_template,
                "memory":st.session_state.conversation_memory,
                }
            )
    #處理使用者輸入並即時生成回覆,接受使用者提問，呼叫問答鏈獲得回答，並逐字模擬輸入效果即時顯示回答。
    if user_input := st.chat_input("You:",key="user_input"):
        user_message={"role":"user","message":user_input}
        st.session_state.chat_history.append(user_message)
        with st.chat_message("user"):
            st.markdown(user_input)

        with st.chat_message("assistant"):
            with st.spinner("助理正在輸入..."):
                response = st.session_state.qa_chain(user_input)
            message_placeholder = st.empty()
            full_response=""
            for chunk in response['result'].split():
                full_response += chunk + ""
                time.sleep(0.05)
                #Add a blinking cursor to simulate typing
                message_placeholder.markdown(full_response + "▌")
            message_placeholder.markdown(full_response)
        chatbot_message = {"role":"assistant","message":response['result']}
        st.session_state.chat_history.append(chatbot_message)

else:
    st.write("請上傳一個PDF檔案來開始 ChatPDF")














再来继续在Anaconda Prompt执行python -m streamlit run 您的程序名称。这里要注意streamlit版本，执行语法可能会有差异。

2.   打开Anaconda Prompt切换到上面python的脚本位置，执行cd python的脚本位置指令。


> cd C:\Users\joyce


3.   你的第一个本地端自然语言对话程式就完成。


> python -m streamlit run documChatbot.py